<a href="https://colab.research.google.com/github/GVGunasekhar/Binance-trade-analysis/blob/main/Internshala_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. Data Exploration and Cleaning**

In [13]:
import pandas as pd
import csv

#Step 1: Load data with adjusted quoting behavior and error handling
file_path = '/content/TRADES_CopyTr_90D_ROI.csv'
# By default, quoting is set to csv.QUOTE_MINIMAL (0), which only quotes fields containing special characters.
# Try changing to csv.QUOTE_ALL (1) to quote all fields and prevent the error.
df = pd.read_csv(file_path, quoting=csv.QUOTE_MINIMAL, on_bad_lines='warn', engine='python', dtype=object)
# or df = pd.read_csv(file_path, on_bad_lines='warn', engine='python', dtype=object)


# Step 2: Inspect the first few rows and data types to identify problematic columns
print("First few rows of the dataset:")
print(df.head(10))  # Display the first 10 rows to get a better sense of the data

print("\nData Types:")
print(df.dtypes)

# Step 3: Identify the problematic columns based on data types and values
# This step helps in isolating columns where conversion or content is an issue.

# Step 4: Convert specific columns to numeric if necessary
# Replace 'column1', 'column2' with actual numeric column names.
# Use a try-except block to catch errors and continue execution for analysis.
numeric_columns = ['column1', 'column2']  # Example placeholder, replace with actual column names
for col in numeric_columns:
    try:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    except KeyError:
        print(f"Column '{col}' not found in the dataset.")

# Step 5: Check for missing values created by coercion
print("\nMissing Values after converting to numeric:")
print(df.isnull().sum())

# Step 6: Handle missing values (if any)
# Fill missing numeric values with the mean of their respective columns
df.fillna(df.mean(numeric_only=True), inplace=True)

# Check missing values again after handling
print("\nMissing Values after filling with mean:")
print(df.isnull().sum())

# Step 7: Optional - Save the cleaned dataset
df.to_csv('cleaned_trade_data.csv', index=False)

# Optional: Display the cleaned dataset information
print("\nCleaned Dataset Information:")
print(df.info())

# Optional: Show a few rows of the cleaned data
print("\nFirst few rows of the cleaned dataset:")
print(df.head())


First few rows of the dataset:
              Port_IDs                                      Trade_History
0  4013978721149559808  [{'time': 1718980200000, 'symbol': 'JASMYUSDT'...
1  3988205565930333696  [{'time': 1718959919000, 'symbol': 'BTCUSDT', ...
2  4031181049693676544  [{'time': 1718979513000, 'symbol': 'CFXUSDT', ...
3  4023697881318718465  [{'time': 1718641182000, 'symbol': 'DOGEUSDT',...
4  4041804592937345281  [{'time': 1718977017000, 'symbol': 'ARUSDT', '...
5  4031173908980084736  [{'time': 1718983817000, 'symbol': 'BTCUSDT', ...
6  3998572645139652353  [{'time': 1718962634000, 'symbol': 'BTCUSDT', ...
7  4031451549482615297  [{'time': 1718985618000, 'symbol': 'ORDIUSDT',...
8  4033639786957934336  [{'time': 1718964015000, 'symbol': 'BTCUSDT', ...
9  4029422834086627072  [{'time': 1718781210000, 'symbol': 'ETHUSDT', ...

Data Types:
Port_IDs         object
Trade_History    object
dtype: object
Column 'column1' not found in the dataset.
Column 'column2' not found in the dat

Skipping line 2: field larger than field limit (131072)
Skipping line 3: field larger than field limit (131072)
Skipping line 4: field larger than field limit (131072)
Skipping line 5: field larger than field limit (131072)
Skipping line 6: field larger than field limit (131072)
Skipping line 7: field larger than field limit (131072)
Skipping line 8: field larger than field limit (131072)
Skipping line 9: field larger than field limit (131072)
Skipping line 10: field larger than field limit (131072)
Skipping line 11: field larger than field limit (131072)
Skipping line 13: field larger than field limit (131072)
Skipping line 14: field larger than field limit (131072)
Skipping line 15: field larger than field limit (131072)
Skipping line 17: field larger than field limit (131072)
Skipping line 19: field larger than field limit (131072)
Skipping line 23: field larger than field limit (131072)
Skipping line 24: field larger than field limit (131072)
Skipping line 25: field larger than fie

#**2. Feature Engineering**

In [21]:
import pandas as pd

# Check the available columns in the DataFrame
print("Available columns in the DataFrame:")
print(df.columns)

# Assuming 'side' and 'positionSide' are columns that classify the trades,
# 'quantity' is money in the trade, 'qty' is the coin amount, 'realizedProfit' is the PnL

# If 'realizedProfit' is not present, check for its alternative names or add it to df if possible.

# Feature Engineering

# Step 1: Add win/lose classification
# First, let's ensure the column 'realizedProfit' exists or correct the column name
if 'realizedProfit' in df.columns:
    df['Win_Position'] = df['realizedProfit'].apply(lambda x: 1 if x > 0 else 0)
else:
    print("Column 'realizedProfit' not found. Please check the column name.")
    # Instead of sys.exit(), handle the missing column gracefully
    # Option 1: Create a placeholder column with default values (e.g., 0)
    df['realizedProfit'] = 0  # Or any other appropriate default value
    df['Win_Position'] = 0 # Create corresponding Win_Position column
    # Option 2: Skip further processing steps that depend on 'realizedProfit' and print a message
    # print("Skipping feature engineering steps dependent on 'realizedProfit'.")

# Step 2: Add a column for trade duration (using timestamp if available)
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # Ensure timestamp is in datetime format
    df = df.sort_values(by=['Port_IDs', 'timestamp'])  # Sort by account and time
    df['Trade_Duration'] = df.groupby('Port_IDs')['timestamp'].diff().dt.total_seconds().fillna(0)
else:
    print("Column 'timestamp' not found. Skipping Trade_Duration calculation.")

# Step 3: Calculate other metrics - example of ROI based on realizedProfit
if 'quantity' in df.columns:
    df['ROI'] = df['realizedProfit'] / df['quantity']
else:
    print("Column 'quantity' not found. Skipping ROI calculation.")

# Step 4: Calculate total positions and win positions per account
# Check if 'Port_IDs' and 'Win_Position' columns exist
if 'Port_IDs' in df.columns and 'Win_Position' in df.columns:
    account_summary = df.groupby('Port_IDs').agg(
        Total_Positions=('Port_IDs', 'count'),
        Win_Positions=('Win_Position', 'sum'),
        PnL=('realizedProfit', 'sum')
    ).reset_index()

    # Win Rate = Win Positions / Total Positions
    account_summary['Win_Rate'] = account_summary['Win_Positions'] / account_summary['Total_Positions']

    # Preview the feature engineered data
    print("\nAccount Summary:")
    print(account_summary.head())
else:
    print("Necessary columns 'Port_IDs' or 'Win_Position' not found. Cannot calculate account summary.")

Available columns in the DataFrame:
Index(['Port_IDs', 'Trade_History'], dtype='object')
Column 'realizedProfit' not found. Please check the column name.
Column 'timestamp' not found. Skipping Trade_Duration calculation.
Column 'quantity' not found. Skipping ROI calculation.

Account Summary:
              Port_IDs  Total_Positions  Win_Positions  PnL  Win_Rate
0  3941019213896463617                1              0    0       0.0
1  3977234346014419201                1              0    0       0.0
2  3981810242465907713                1              0    0       0.0
3  3983074113875692800                1              0    0       0.0
4  3988205565930333696                1              0    0       0.0


#**3. Metrics Calculation**

In [22]:
# Import necessary libraries for metrics calculation
import numpy as np

# Example: Calculate Sharpe Ratio (assuming we have daily returns)
def calculate_sharpe_ratio(returns, risk_free_rate=0.0):
    return np.mean(returns - risk_free_rate) / np.std(returns)

# Calculate Sharpe Ratio for each account (using 'realizedProfit' as returns proxy)
df['Daily_Return'] = df.groupby('Port_IDs')['realizedProfit'].pct_change().fillna(0)
sharpe_ratios = df.groupby('Port_IDs')['Daily_Return'].apply(calculate_sharpe_ratio).reset_index(name='Sharpe_Ratio')

# Maximum Drawdown calculation (assuming 'realizedProfit' represents cumulative return over time)
def calculate_max_drawdown(returns):
    cumulative_returns = returns.cumsum()
    drawdowns = cumulative_returns - cumulative_returns.cummax()
    return drawdowns.min()

mdd = df.groupby('Port_IDs')['realizedProfit'].apply(calculate_max_drawdown).reset_index(name='MDD')

# Merge all calculated metrics into a single DataFrame
metrics = account_summary.merge(sharpe_ratios, on='Port_IDs').merge(mdd, on='Port_IDs')

# Preview the metrics DataFrame
print(metrics.head())


              Port_IDs  Total_Positions  Win_Positions  PnL  Win_Rate  \
0  3941019213896463617                1              0    0       0.0   
1  3977234346014419201                1              0    0       0.0   
2  3981810242465907713                1              0    0       0.0   
3  3983074113875692800                1              0    0       0.0   
4  3988205565930333696                1              0    0       0.0   

   Sharpe_Ratio  MDD  
0           NaN    0  
1           NaN    0  
2           NaN    0  
3           NaN    0  
4           NaN    0  


<ipython-input-22-1a1d1e28eb90>:6: RuntimeWarning: invalid value encountered in scalar divide
  return np.mean(returns - risk_free_rate) / np.std(returns)


#**4. Scoring System for Ranking**

In [23]:
# Example: Define weights for each metric (you can adjust based on domain knowledge)
weights = {
    'PnL': 0.4,
    'Sharpe_Ratio': 0.3,
    'Win_Rate': 0.2,
    'MDD': 0.1  # MDD negatively impacts the score, so it might have lower weight
}

# Normalize MDD (since it's a negative metric)
metrics['MDD'] = -metrics['MDD']  # Invert to treat as positive impact on the score

# Calculate a weighted score for each account
metrics['Score'] = (
    metrics['PnL'] * weights['PnL'] +
    metrics['Sharpe_Ratio'] * weights['Sharpe_Ratio'] +
    metrics['Win_Rate'] * weights['Win_Rate'] +
    metrics['MDD'] * weights['MDD']
)

# Rank accounts by score
metrics['Rank'] = metrics['Score'].rank(ascending=False)

# Preview the ranked accounts
print(metrics.sort_values(by='Score', ascending=False).head(20))

# Save top 20 ranked accounts
top_20_accounts = metrics.sort_values(by='Score', ascending=False).head(20)
top_20_accounts.to_csv('top_20_accounts.csv', index=False)


               Port_IDs  Total_Positions  Win_Positions  PnL  Win_Rate  \
0   3941019213896463617                1              0    0       0.0   
1   3977234346014419201                1              0    0       0.0   
2   3981810242465907713                1              0    0       0.0   
3   3983074113875692800                1              0    0       0.0   
4   3988205565930333696                1              0    0       0.0   
5   3998572645139652353                1              0    0       0.0   
6   4013978721149559808                1              0    0       0.0   
7   4017110277719148289                1              0    0       0.0   
8   4023697433751327232                1              0    0       0.0   
9   4023697881318718465                1              0    0       0.0   
10  4026179081117855488                1              0    0       0.0   
11  4028427053699127040                1              0    0       0.0   
12  4029299190618134272               

#**5. Documentation**

**Methodology:**

**Data Exploration:** We loaded the data and handled missing values.

**Feature Engineering:**We created features for Win/Loss classification, ROI, PnL, trade duration, etc.

**Metrics Calculation:** We calculated ROI, PnL, Sharpe Ratio, MDD, Win Rate, and Total Positions.

**Ranking Algorithm:** We created a scoring system with weighted metrics to rank accounts.

**Findings:**
The top 20 accounts were ranked based on the score that combined PnL, Sharpe Ratio, Win Rate, and MDD.

**Assumptions:**
Missing values were handled by filling with column means.
The Sharpe Ratio used realizedProfit as a proxy for returns and assumed a risk-free rate of 0.
Maximum Drawdown (MDD) was calculated based on cumulative profits.
